# How to use Bedrock with LiteLLM
This notebook demonstrates how to use Bedrock with LiteLLM. Making calls to various Large Language Models in Amazon Bedrock using LiteLLM SDK is simplified in this notebook.

Customers implementing GenAI workloads at scale in production look for Load-balancing, routing and budget controls. This is easily achieved using an sdk such as LiteLLM which offers a range of features including load-balancing, budget and routing.

In this example, you will learn how to make calls to Bedrock API using LiteLLM.
1. Overview - The notebook demonstrates how to use Amazon Bedrock with LiteLLM, simplifying calls to various Large Language Models (LLMs) in Amazon Bedrock using the LiteLLM SDK.
    1. What are we demonstrating - We're showing how to make API calls to Amazon Bedrock using LiteLLM, including basic completions, function calling, using guardrails, and streaming responses.
    2. What use case - This is useful for customers implementing GenAI workloads at scale in production who need load-balancing, routing, and budget controls, which LiteLLM offers.
    3. What will you learn:
        - How to set up and use LiteLLM with Amazon Bedrock
        - Making basic completion calls to Bedrock models
        - Using function calling with Bedrock and LiteLLM
        - Implementing Bedrock Guardrails with LiteLLM
        - Using streaming for responses
2. What is the architectural pattern and why we select this - The pattern uses LiteLLM as an abstraction layer over Amazon Bedrock. This is chosen because it simplifies API calls and provides additional features like load-balancing and budget controls.
    [User Application] -> [LiteLLM SDK] -> [Amazon Bedrock API] -> [Various LLMs]
3. What are the libraries to install - litellm, boto3
4. What model did we choose and why this model - The notebook uses "bedrock/anthropic.claude-3-haiku-20240307-v1:0", which is Claude 3 Haiku model from Anthropic on Amazon Bedrock. This model is likely chosen for its capabilities and availability on Bedrock.
5. Every cell needs to have a markup - The notebook does include markdown cells explaining each section. The code is structured into sections for different functionalities (basic usage, function calling, guardrails, streaming).

Let's start by importing the libraries used in this example

In [ ]:
!python3 -m pip install litellm --quiet
!python3 -m pip install boto3 --quiet # LiteLLM requires boto3 to be installed for Bedrock APi requests

### Setup AWS Credentials
We'll now setup AWS Environment variables to use a region and a profile/iam_role as below

In [ ]:
# set env
# os.environ["AWS_ACCESS_KEY_ID"] = ""
# os.environ["AWS_SECRET_ACCESS_KEY"] = ""
# os.environ["AWS_REGION_NAME"] = ""

from litellm import completion

response = completion(
    model="bedrock/anthropic.claude-3-haiku-20240307-v1:0",
    messages=[{ "content": "Hello, how are you?","role": "user"}],
    max_tokens=1000,
    temperature=0.7
    # aws_profile_name=""
)
print (response.choices[0].message.content)

You can also specify a region in the call as follows:

In [ ]:
from litellm import completion

response = completion(
    model="bedrock/anthropic.claude-3-haiku-20240307-v1:0",
    messages=[{ "content": "Hello, how are you?","role": "user"}],
    max_tokens=1000,
    temperature=0.7,
    aws_region_name="us-west-2"
)
print (response.choices[0].message.content)

## How to use Function Calling with Bedrock + LiteLM

In [ ]:
from litellm import completion

tools =  [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]
messages = [{"role": "user", "content": "What's the weather like in Boston today?"}]

response = completion(
    model="bedrock/anthropic.claude-3-haiku-20240307-v1:0",
    messages=messages,
    tools=tools,
    tool_choice="auto",
)

# Add any assertions, here to check response args
print(response)
assert isinstance(response.choices[0].message.tool_calls[0].function.name, str)
assert isinstance(
    response.choices[0].message.tool_calls[0].function.arguments, str
)

## How to use Bedrock Guardrails with LiteLM
1. We first setup a new GuardRail in AWS Console --> Bedrock --> GuardRails.
2. Once a Guardrail is setup, fetch the `Guardrail ID and Version` and replace the `$guardrailIdentifier` & `guardrailVersion`in the code below.

In [ ]:
import os
from litellm import completion

response = completion(
    model="bedrock/anthropic.claude-3-haiku-20240307-v1:0",
    messages=[{ "content": "Who were the winners of Olympics 2021 Swimming?","role": "user"}],
    max_tokens=100,
    temperature=0.7,
    guardrailConfig={
        "guardrailIdentifier": "REPLACE_ME", # The identifier (ID) for the guardrail.
        "guardrailVersion": "REPLACE_ME", # The version of the guardrail.
        "trace": "enabled", # The trace behavior for the guardrail. Can either be "disabled" or "enabled"
    },
)
print (response.choices[0].message.content)

## Use Streaming

In [ ]:
from litellm import completion

response = completion(
    model="bedrock/anthropic.claude-3-haiku-20240307-v1:0",
    messages=[{"role": "user", "content": "Who were the winners of Olympics 2021 Swimming?"}],
    max_tokens=100,
    temperature=0.7,
    stream=True
)
for chunk in response:
    print(chunk.choices[0].delta.content or "")